In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotnine as p9
import os

from lib.lib import Import_data

path = Import_data()
# get all files in the directory
files = os.listdir(path)
print(files)

/Users/mathiasraa/Desktop/ntnu/eit/backend/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


Path to dataset files: /Users/mathiasraa/.cache/kagglehub/datasets/arashnic/earthquake-magnitude-damage-and-impact/versions/6
['csv_household_resources.csv', 'csv_building_structure.csv', 'ward_vdcmun_district_name_mapping.csv', 'mapping.csv', 'csv_building_ownership_and_use.csv', 'csv_building_damage_assessment.csv', 'csv_household_demographics.csv', 'csv_individual_demographics.csv', 'csv_household_earthquake_impact.csv']


In [75]:
building_types = {
    "Traditional Bamboo": {
        "description": "Simple bamboo structure with timber framing",
        "foundation_type": "Bamboo/Timber",
        "roof_type": "Bamboo/Timber-Light roof",
        "ground_floor_type": "Mud",
        "other_floor_type": "Not applicable",
        "has_superstructure_timber": 1,
        "has_superstructure_bamboo": 1,
        "base_cost": 909,  # 1 LAKH NPR
        "src": "/building_types/bamboo.png",
    },
    "Basic Stone/Brick": {
        "description": "Stone/brick home with light roof and mud mortar",
        "foundation_type": "Mud mortar-Stone/Brick",
        "roof_type": "Bamboo/Timber-Light roof",
        "ground_floor_type": "Mud",
        "other_floor_type": "TImber/Bamboo-Mud",
        "has_superstructure_mud_mortar_stone": 1,
        "base_cost": 1455,  # NPR 100,000 to NPR 500,000
        "src": "/building_types/brick_mud.png",
    },
    "Improved Stone/Brick": {
        "description": "Stone/brick home with heavy timber roof for better stability",
        "foundation_type": "Mud mortar-Stone/Brick",
        "roof_type": "Bamboo/Timber-Heavy roof",
        "ground_floor_type": "Brick/Stone",
        "other_floor_type": "TImber/Bamboo-Mud",
        "has_superstructure_mud_mortar_stone": 1,
        "base_cost": 3637,
        "src": "/building_types/improved_brick.png",
    },
    "Modern Concrete": {
        "description": "Reinforced concrete structure with modern materials",
        "foundation_type": "RC",
        "roof_type": "RCC/RB/RBC",
        "ground_floor_type": "RC",
        "other_floor_type": "RCC/RB/RBC",
        "has_superstructure_cement_mortar_brick": 1,
        "base_cost": 7275,
        "src": "/building_types/modern_concrete.png",
    },
}


In [4]:
building_structure = pd.read_csv(path + "/csv_building_structure.csv")

In [85]:
building = building_types["Basic Stone/Brick"]

foundation_type = building_structure["foundation_type"] == building["foundation_type"]
roof_type = building_structure["roof_type"] == building["roof_type"]
ground_floor_type = building_structure["ground_floor_type"] == building["ground_floor_type"]
other_floor_type = building_structure["other_floor_type"] == building["other_floor_type"]
has_superstructure_mud_mortar_stone = building_structure["has_superstructure_mud_mortar_stone"] == building.get("has_superstructure_mud_mortar_stone", 0)
has_superstructure_timber = building_structure["has_superstructure_timber"] == building.get("has_superstructure_timber", 0)
has_superstructure_bamboo = building_structure["has_superstructure_bamboo"] == building.get("has_superstructure_bamboo", 0)
has_superstructure_cement_mortar_brick = building_structure["has_superstructure_cement_mortar_brick"] == building.get("has_superstructure_cement_mortar_brick", 0)

In [86]:
damage_grades = building_structure[
    foundation_type
    & roof_type
    & ground_floor_type
    & other_floor_type
    & has_superstructure_mud_mortar_stone
    & has_superstructure_timber
    & has_superstructure_bamboo
    & has_superstructure_cement_mortar_brick
]["damage_grade"]

# Weighted average by count damage grade
damage_grade = damage_grades.value_counts().sort_index()
damage_grade.index = np.array([1, 2, 3, 4, 5])
damage_grade = damage_grade / damage_grade.sum()
damage_grade = damage_grade * damage_grade.index
damage_grade = damage_grade.sum()
print(damage_grade)

4.005705185903206


In [87]:
damage_grades.value_counts()

damage_grade
Grade 5    90922
Grade 4    64998
Grade 3    37935
Grade 2    16619
Grade 1     6171
Name: count, dtype: int64